In [ ]:
!pip install geopy hvplot optuna plotly pytz joblib tqdm bokeh

In [1]:
# Importing packages
import copy
import os
import sys

import geopy.distance
import hvplot.pandas
import numpy as np
import optuna
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pytz
import tensorflow as tf
from bokeh.sampledata.penguins import data as df
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tqdm import tqdm

2024-08-24 11:43:39.747635: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-24 11:43:39.751933: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-24 11:43:39.802310: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-24 11:43:39.802359: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-24 11:43:39.802387: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [13]:
# Rain Info
flag_use_rainy_days_only = True
path_rain_measurements = "../../data/israel_meteorological/rain_202408141745.csv"
rain_stations_measurements = pd.read_csv(path_rain_measurements)
rain_stations_measurements["date"] = pd.to_datetime(
    rain_stations_measurements["date"], format="%d/%m/%Y"
)
rain_stations_measurements

,station,date,rain_amount,rain_code
0,להבות חביבה 09/1994-07/2024,2017-01-01,1.1,5
1,כפר הרואה 09/1978-07/2024,2017-01-01,1.5,0
2,עין החורש 09/2004-07/2024,2017-01-01,1.4,0
3,עין החורש מאוישת 09/1975-07/2024,2017-01-01,1.4,0
4,מעברות 09/2006-07/2024,2017-01-01,1.2,0
...,...,...,...,...
16723,גן שלמה 09/1939-07/2024,2022-12-26,28.5,0
16724,גבעת ברנר 09/1984-07/2024,2022-12-26,40.0,0
16725,שעלבים 09/1951-07/2024,2022-12-26,23.5,0
16726,ראש העין - מקורות 10/1989-07/2024,2022-12-27,0.0,9


In [15]:
# Filter unwanted rain measurements
# Rain codes info are here: "C:\Users\Yanay\Desktop\yanay\master\meteorological stations\קוד גשם.pdf"
# 3 - unknown
rain_stations_measurements = rain_stations_measurements.query("`rain_code` != 3 and rain_amount > 0").reset_index(drop=True)
rain_stations_measurements

,station,date,rain_amount,rain_code
0,להבות חביבה 09/1994-07/2024,2017-01-01,1.1,5
1,כפר הרואה 09/1978-07/2024,2017-01-01,1.5,0
2,עין החורש 09/2004-07/2024,2017-01-01,1.4,0
3,עין החורש מאוישת 09/1975-07/2024,2017-01-01,1.4,0
4,מעברות 09/2006-07/2024,2017-01-01,1.2,0
...,...,...,...,...
16094,נצר סרני 09/1977-07/2024,2022-12-26,29.1,0
16095,גן שלמה 09/1939-07/2024,2022-12-26,28.5,0
16096,גבעת ברנר 09/1984-07/2024,2022-12-26,40.0,0
16097,שעלבים 09/1951-07/2024,2022-12-26,23.5,0


In [10]:
rainy_dates = rain_stations_measurements["date"].dt.date.unique()
print(f"number of rainy days: {len(rainy_dates)}")
rain_stations_measurements.hvplot.hist("date", bins=364).opts(
    title="Number of stations collected rain"
)

number of rainy days: 536


:Histogram   [date]   (Count)

In [ ]:
# path
sensor_location_path = "input/United_processed_data/sensors_location"
all_data_path = "input/United_processed_data/united_data"

In [16]:
!pwd

/home/jovyan/work/src/notebooks


In [19]:
pp = "/home/jovyan/work/data/NREL/processed/1930179_31.89_35.46_2017.pkl"
# pd.read_pickle(pp)
import pickle
with open(pp, 'rb') as file:
    cur_df = pickle.load(file)

ModuleNotFoundError: No module named 'numpy._core'

In [ ]:
sensor_location = pd.read_pickle(sensor_location_path)

In [ ]:
lon_vec = np.sort(np.unique(sensor_location["Longitude"]))
lat_vec = np.sort(np.unique(sensor_location["Latitude"]))

coords_1 = (lat_vec[1], lon_vec[1])
coords_2 = (lat_vec[0], lon_vec[1])
coords_3 = (lat_vec[0], lon_vec[0])

lat_dist = geopy.distance.geodesic(coords_1, coords_2).km
lon_dist = geopy.distance.geodesic(coords_2, coords_3).km

print("num of sensors: " + str(len(lat_vec)) + "x" + str(len(lon_vec)))
print("X distance = " + str(lon_dist))
print("Y distance = " + str(lat_dist))
print("total X distance = " + str((len(lon_vec) - 1) * lon_dist))
print("total Y distance = " + str((len(lat_vec) - 1) * lat_dist))
print("Max Longitude = " + str(max(lon_vec)))
print("Min Longitude = " + str(min(lon_vec)))
print("Max Latitude = " + str(max(lat_vec)))
print("Min Latitude = " + str(min(lat_vec)))

tested_sensor_location = sensor_location.copy()

tested_sensor_location.hvplot.points(
    x="Longitude",
    y="Latitude",
    cmap="bwr",
    geo=True,
    xlim=(34.5, 35.2),
    ylim=(31.7, 32.5),
    tiles="CartoLight",
)

In [ ]:
def merge_vals_with_location(sensor_location, df_vals, col_name):
    # Extract the sensor ID from the column names in df_vals
    sensor_ids = df_vals.columns.str.split("_").str[2]
    sensor_values = df_vals.values.flatten()
    # Create a DataFrame with the 'ID' column
    id_val_df = pd.DataFrame({"ID": sensor_ids, col_name: sensor_values})
    merged_df = id_val_df.merge(sensor_location, on="ID")

    return merged_df

In [ ]:
def LSTMDataPreparation(
    all_data_path,
    hours_to_use,
    num_taps_to_use,
    num_taps_in_future_to_predict,
    flag_use_rainy_days_only,
):
    united_data = pd.read_pickle(all_data_path)

    if flag_use_rainy_days_only:
        united_data = united_data[united_data["Time"].dt.date.isin(rainy_dates)]

    data_df = (
        pd.DataFrame()
    )  # Initialize an empty DataFrame for concatenating cur_data_df
    target_df = (
        pd.DataFrame()
    )  # Initialize an empty DataFrame for concatenating cur_target_df

    united_data = united_data.query(
        f"Time.dt.hour >= {hours_to_use[0]} and Time.dt.hour <= {hours_to_use[1]}"
    ).reset_index(drop=True)

    # Drop Unused data for training
    GHI_to_drop_columns = united_data.columns[
        united_data.columns.str.contains("GHI")
        & ~united_data.columns.str.contains("norm")
    ]
    united_data = united_data.drop(GHI_to_drop_columns, axis=1)
    united_data = united_data.drop(["csi_haurwitz"], axis=1)

    grouped_united_data = united_data.groupby(united_data["Time"].dt.date)
    X, y = [], []
    X_time, y_time = [], []
    for group_name, group_data in tqdm(grouped_united_data, desc="Processing groups"):
        time_data = group_data["Time"]
        input_data = group_data.drop(["Time"], axis=1)
        output_data = input_data.drop(["azimuth", "altitude"], axis=1)

        input_columns = input_data.columns
        output_columns = output_data.columns

        ################ Normalize azimuth/altitude for LSTM ################
        time_data.azimuth = 180
        time_data.altitude = 45
        #####################################################################

        # data = group_data
        n_samples, n_features = input_data.shape

        for i in range(n_samples - num_taps_to_use - num_taps_in_future_to_predict + 1):
            # Input sequence
            input_seq = input_data.iloc[i : i + num_taps_to_use].to_numpy()
            input_seq = np.expand_dims(
                input_seq, axis=0
            )  # Add a new dimension for num_samples
            # Output sequence (target)
            output_seq = output_data.iloc[
                i
                + num_taps_to_use : i
                + num_taps_to_use
                + num_taps_in_future_to_predict
            ].to_numpy()
            output_seq = np.expand_dims(
                output_seq, axis=0
            )  # Add a new dimension for num_samples

            # Time sequences
            input_time_seq = time_data.iloc[i : i + num_taps_to_use].to_numpy()
            input_time_seq = np.expand_dims(
                input_time_seq, axis=0
            )  # Add a new dimension for num_samples
            output_time_seq = time_data.iloc[
                i
                + num_taps_to_use : i
                + num_taps_to_use
                + num_taps_in_future_to_predict
            ].to_numpy()
            output_time_seq = np.expand_dims(
                output_time_seq, axis=0
            )  # Add a new dimension for num_samples

            X.append(input_seq)
            y.append(output_seq)
            X_time.append(input_time_seq)
            y_time.append(output_time_seq)

        # Convert the lists to numpy arrays
    X = np.concatenate(X, axis=0)
    y = np.concatenate(y, axis=0)
    X_time = np.concatenate(X_time, axis=0)
    y_time = np.concatenate(y_time, axis=0)

    # Split data into training and validation sets
    train_ratio = 0.8  # 80% for training, 20% for validation
    train_size = int(train_ratio * X.shape[0])
    x_train = X[:train_size, :, :]
    y_train = y[:train_size, :, :]
    x_test = X[train_size:, :, :]
    y_test = y[train_size:, :, :]

    # Time vecs:
    x_train_time = X_time[:train_size, :]
    y_train_time = y_time[:train_size, :]
    x_test_time = X_time[train_size:, :]
    y_test_time = y_time[train_size:, :]

    return (
        x_train,
        y_train,
        x_test,
        y_test,
        x_train_time,
        y_train_time,
        x_test_time,
        y_test_time,
        input_columns,
        output_columns,
    )

## Trying to find best output using optuna

In [ ]:
import numpy as np
import optuna
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2


# Define the objective function to be optimized
def objective(trial):
    # Define the search space for hyperparameters
    layer_size = trial.suggest_int("layer_size", 32, 128)
    dropout_rate = trial.suggest_float("dropout_rate", 0.2, 0.5)
    l2_reg = trial.suggest_float("l2_reg", 1e-5, 0.1, log=True)

    # Build the model using the suggested hyperparameters
    model = Sequential()
    model.add(
        LSTM(
            layer_size,
            activation="relu",
            input_shape=(seq_length, n_features_input),
            dropout=dropout_rate,
            kernel_regularizer=l2(l2_reg),
        )
    )
    model.add(RepeatVector(pred_length))
    model.add(
        LSTM(
            layer_size,
            activation="relu",
            return_sequences=True,
            dropout=dropout_rate,
            kernel_regularizer=l2(l2_reg),
        )
    )
    model.add(TimeDistributed(Dense(n_features_output, kernel_regularizer=l2(l2_reg))))
    model.compile(optimizer="adam", loss="mse")

    # Train the model and return the validation loss as the objective value
    model.fit(
        x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2
    )
    val_loss = model.evaluate(x_test, y_test)

    return val_loss

In [ ]:
# Data preperation
# Configs for the data
#####################################################################
num_taps_to_use = 8  # every tap 15 min
num_taps_in_future_to_predict = 4  # should be at least 1
hours_to_use = [5, 13]  # This is in UTC time zone
#####################################################################
(
    x_train,
    y_train,
    x_test,
    y_test,
    x_train_time,
    y_train_time,
    x_test_time,
    y_test_time,
    input_columns,
    output_columns,
) = LSTMDataPreparation(
    all_data_path,
    hours_to_use,
    num_taps_to_use,
    num_taps_in_future_to_predict,
    flag_use_rainy_days_only,
)

In [ ]:
seq_length = num_taps_to_use
pred_length = num_taps_in_future_to_predict
n_features_input = x_train.shape[2]
n_features_output = y_train.shape[2]
epochs = 15
batch_size = 16

flag_explore_using_optuna = True

if flag_explore_using_optuna:
    # Create a study object and optimize the objective function
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=100)

    # Print the optimization results
    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial
    print("  Value: ", trial.value)
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    # Save the study to a local file
    joblib.dump(study, "ed_lstm_trial.pkl")

In [ ]:
# Define other constants for training
flag_train_model = False
flag_get_params_from_best_trial = True

if flag_train_model:
    if flag_get_params_from_best_trial:
        ed_lstm_trial = joblib.load("ed_lstm_trial.pkl")
        best_layer_size = ed_lstm_trial.best_params["layer_size"]
        best_dropout_rate = ed_lstm_trial.best_params["dropout_rate"]
        best_l2_reg = ed_lstm_trial.best_params["l2_reg"]
    else:
        # Use the best hyperparameters to build the model
        best_layer_size = 110
        best_dropout_rate = 0.20461123192283945
        best_l2_reg = 1.835024618408312e-05

    # Build the model using the best hyperparameters
    best_model = Sequential()
    best_model.add(
        LSTM(
            best_layer_size,
            activation="relu",
            input_shape=(seq_length, n_features_input),
            dropout=best_dropout_rate,
            kernel_regularizer=l2(best_l2_reg),
        )
    )
    best_model.add(RepeatVector(pred_length))
    best_model.add(
        LSTM(
            best_layer_size,
            activation="relu",
            return_sequences=True,
            dropout=best_dropout_rate,
            kernel_regularizer=l2(best_l2_reg),
        )
    )
    best_model.add(
        TimeDistributed(Dense(n_features_output, kernel_regularizer=l2(best_l2_reg)))
    )
    best_model.compile(optimizer="adam", loss="mse")

    # Define callbacks
    early_stopping = EarlyStopping(
        monitor="val_loss", patience=5, restore_best_weights=True
    )
    model_checkpoint = ModelCheckpoint(
        "best_model_checkpoint.h5", monitor="val_loss", save_best_only=True
    )

    # Train the model using the best hyperparameters
    history = best_model.fit(
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.2,
        callbacks=[early_stopping, model_checkpoint],
    )

    # Evaluate the trained model on test data
    test_loss = best_model.evaluate(x_test, y_test)
    print("Test loss with the best model:", test_loss)

    # Save the best model using the native Keras format
    best_model.save("best_model.keras")
    print("Best model saved.")
    
    # Visualize convergence using Plotly
    fig = px.line(
    history.history, y=["loss", "val_loss"], labels={"x": "Epoch", "y": "Loss"})
    fig.update_layout(title="Training and Validation Loss Convergence")
    fig.show()

else:
    # Load the best model from the checkpoint (if needed)
    best_model = tf.keras.models.load_model("best_model_checkpoint.h5")

Best trial:<br>
  Value:  0.04920314997434616<br>
  Params: <br>
    layer_size: 110<br>
    dropout_rate: 0.20461123192283945<br>
    l2_reg: 1.835024618408312e-0<br>

In [ ]:
def calc_persistence_predictions(data, data_columns, num_taps_in_future_to_predict):
    GHI_columns_idx = input_columns.str.contains("GHI")
    presistence_value = x_test[:, -1, GHI_columns_idx]

    # Repeat along a new axis
    presistence_pred = np.repeat(
        presistence_value[:, np.newaxis, :],
        repeats=num_taps_in_future_to_predict,
        axis=1,
    )
    return presistence_pred

In [ ]:
def rmse_model_errors(pred, targets):
    model_error = pred - targets
    model_error_square = model_error**2
    rmse_mean_date = np.sqrt(np.mean(model_error_square, axis=0))
    rmse_mean_taps = np.sqrt(np.mean(model_error_square, axis=1))
    rmse_mean_sensors = np.sqrt(np.mean(model_error_square, axis=2))

    return rmse_mean_date, rmse_mean_taps, rmse_mean_sensors

In [ ]:
persistence_test = calc_persistence_predictions(
    x_test, input_columns, num_taps_in_future_to_predict
)
(
    rmse_mean_date_persistence,
    rmse_mean_taps_persistence,
    rmse_mean_sensors_persistence,
) = rmse_model_errors(persistence_test, y_test)

model_pred_test = best_model(x_test)
(rmse_mean_date_lstm, rmse_mean_taps_lstm, rmse_mean_sensors_lstm) = rmse_model_errors(
    model_pred_test, y_test
)

SKILL_mean_date_lstm = 100 * (1 - rmse_mean_date_lstm / rmse_mean_date_persistence)
SKILL_mean_taps_lstm = 100 * (1 - rmse_mean_taps_lstm / rmse_mean_taps_persistence)
SKILL_mean_sensors_lstm = 100 * (
    1 - rmse_mean_sensors_lstm / rmse_mean_sensors_persistence
)

In [ ]:
df_SKILL_mean_date_lstm = pd.DataFrame(SKILL_mean_date_lstm, columns=output_columns)

plots = []
for i in range(df_SKILL_mean_date_lstm.shape[0]):
    df_sensors_lstm_skill = merge_vals_with_location(
        sensor_location, df_SKILL_mean_date_lstm.iloc[[i]], "skill_lstm"
    )
    df_sensors_lstm_skill_plot = df_sensors_lstm_skill.hvplot.points(
        x="Longitude",
        y="Latitude",
        c="skill_lstm",
        geo=True,
        tiles="CartoLight",
        clim=(np.min(SKILL_mean_date_lstm), np.max(SKILL_mean_date_lstm)),
        cmap="jet",
    ).opts(title=f"LSTM skill {(i + 1)*15}")
    plots.append(df_sensors_lstm_skill_plot)

layout = plots[0]
for plot in plots[1:]:
    layout += plot

layout

In [ ]:
df_rmse_mean_date_lstm = pd.DataFrame(rmse_mean_date_lstm, columns=output_columns)

plots = []
for i in range(df_rmse_mean_date_lstm.shape[0]):
    df_sensors_lstm_rmse = merge_vals_with_location(
        sensor_location, df_rmse_mean_date_lstm.iloc[[i]], "rmse_lstm"
    )
    df_sensors_lstm_rmse_plot = df_sensors_lstm_rmse.hvplot.points(
        x="Longitude",
        y="Latitude",
        c="rmse_lstm",
        geo=True,
        tiles="CartoLight",
        clim=(np.min(rmse_mean_date_lstm), np.max(rmse_mean_date_lstm)),
        cmap="jet",
    ).opts(title=f"LSTM rmse {(i + 1)*15}")
    plots.append(df_sensors_lstm_rmse_plot)

layout = plots[0]
for plot in plots[1:]:
    layout += plot

layout

In [ ]:
df_rmse_mean_date_persistence = pd.DataFrame(
    rmse_mean_date_persistence, columns=output_columns
)

plots = []
for i in range(df_rmse_mean_date_persistence.shape[0]):
    df_sensors_persistence_rmse = merge_vals_with_location(
        sensor_location, df_rmse_mean_date_persistence.iloc[[i]], "rmse_persistence"
    )
    df_sensors_persistence_rmse_plot = df_sensors_persistence_rmse.hvplot.points(
        x="Longitude",
        y="Latitude",
        c="rmse_persistence",
        geo=True,
        tiles="CartoLight",
        clim=(np.min(rmse_mean_date_persistence), np.max(rmse_mean_date_persistence)),
        cmap="jet",
    ).opts(title=f"persistence rmse {(i + 1)*15}")
    plots.append(df_sensors_persistence_rmse_plot)

layout = plots[0]
for plot in plots[1:]:
    layout += plot

layout

In [ ]:
# Create a line plot for each column
fig = go.Figure()

# Loop for persistence model
for col_idx in range(rmse_mean_sensors_persistence.shape[1]):
    fig.add_trace(
        go.Scatter(
            x=y_test_time[:, 0],
            y=rmse_mean_sensors_persistence[:, col_idx],
            name=f"persistence {(col_idx + 1)*15}",
            mode="lines+markers",
        )
    )

# Loop for lstm model
for col_idx in range(rmse_mean_sensors_lstm.shape[1]):
    fig.add_trace(
        go.Scatter(
            x=y_test_time[:, 0],
            y=rmse_mean_sensors_lstm[:, col_idx],
            name=f"lstm {(col_idx + 1)*15}",
            line=dict(dash="dash"),
        )
    )

fig.update_layout(
    title="Models rmse Values vs Time", xaxis_title="Time", yaxis_title="rmse Value"
)

fig.show()

In [ ]:
# Create a line plot for each column
fig = go.Figure()

for col_idx in range(rmse_mean_sensors_lstm.shape[1]):
    fig.add_trace(
        go.Scatter(
            x=y_test_time[:, 0],
            y=SKILL_mean_sensors_lstm[:, col_idx],
            name=f"SKILL {(col_idx + 1)*15}",
            mode="lines+markers",
        )
    )


fig.update_layout(
    title="LSTM SKILL vs Time", xaxis_title="Time", yaxis_title="SKILL Value"
)

fig.show()

In [ ]:
agg_metrics_df = pd.DataFrame({
    "lstm_median_skill": df_SKILL_mean_date_lstm.transpose().median().to_list(),
"lstm_min_skill": df_SKILL_mean_date_lstm.transpose().min().to_list(),
"lstm_max_skill": df_SKILL_mean_date_lstm.transpose().max().to_list()
})

fig = go.Figure()

for column in agg_metrics_df.columns:  # Skip the first column 'x'
    fig.add_trace(go.Scatter(x=15*(agg_metrics_df.index+1), y=agg_metrics_df[column], mode='lines', name=column))

# Update layout
fig.update_layout(title='SKILL vs predicted time',
                  xaxis_title='Predicted Time [minutes]',
                  yaxis_title='SKILL')

# Show the plot
fig.show()

agg_metrics_df = pd.DataFrame({
    "lstm_median_rRMSE": df_rmse_mean_date_lstm.transpose().median().to_list(),
"lstm_min_rRMSE": df_rmse_mean_date_lstm.transpose().min().to_list(),
"lstm_max_rRMSE": df_rmse_mean_date_lstm.transpose().max().to_list()
})

fig = go.Figure()

for column in agg_metrics_df.columns:  # Skip the first column 'x'
    fig.add_trace(go.Scatter(x=15*(agg_metrics_df.index+1), y=agg_metrics_df[column], mode='lines', name=column))

# Update layout
fig.update_layout(title='nRMSE vs predicted time',
                  xaxis_title='Predicted Time [minutes]',
                  yaxis_title='nRMSE')

# Show the plot
fig.show()